In [4]:

import sys
import os
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
from numpy import pi, matmul,sqrt,dot,array,zeros,cos,sin,pi,arccos
from torch.utils.data import DataLoader
from func import OR,heatplot,misorientation,ipfread,negsample
from Class import Data,Cluster,Dataset,CNN50a,CNN50b,CNN49a,CNN49b
import torch.nn.functional as F
import matplotlib.patches as patches


In [11]:
feature=["Error","BS"]
pic_size=500
sam_size=49
def create_dataset(root="data/train/",feature=["Orient","MAD"],pic_size=500,sam_size=50):
    corner=torch.load("data/780_corner.pkl")
    bef=[]
    target=[]
    source=[]
    h=w=sam_size
    for date in sorted(os.listdir(root)):
        path=root+date+"/before/"
        data=Data(path)
        data.data["Orient"]=data.data["Orient"].reshape(data.h,data.w,-1)
        for (i,j) in corner[date]:
            app=[]
            for ele in feature:
                if ele=="Orient":
                    app.append(data.data[ele][i:i+h,j:j+w])
                else:   
                    app.append(data.data[ele][i:i+h,j:j+w,np.newaxis])
            bef.append(np.concatenate(app,axis=2))
            source.append([date,(i,j)])
            target.append(0)
        for (i,j) in negsample(corner[date],pic_size,sam_size):
            app=[]
            for ele in feature:
                if ele=="Orient":
                    app.append(data.data[ele][i:i+h,j:j+w])
                else:
                    app.append(data.data[ele][i:i+h,j:j+w,np.newaxis])
            source.append([date,(i,j)])
            bef.append(np.concatenate(app,axis=2))
            target.append(1)
    bef=np.transpose(np.array(bef),(0,3,1,2))
    target=np.array(target)
    return Dataset(bef,target,source)
train=create_dataset("data/train/",feature=feature,pic_size=pic_size,sam_size=sam_size)
test=create_dataset("data/test/",feature=feature,pic_size=pic_size,sam_size=sam_size)

In [32]:
class CNN49a(nn.Module):
    def __init__(self,in_channel,comp):
        super(CNN49a, self).__init__()
        self.batchnorm = nn.BatchNorm2d(in_channel)
        self.relu = nn.ReLU() # activation
        self.maxpool = nn.MaxPool2d(kernel_size=2) 
        self.cnn1 = nn.Conv2d(in_channels=in_channel, out_channels=12, kernel_size=2, stride=1, padding=0) 
        self.cnn2 = nn.Conv2d(in_channels=12, out_channels=18, kernel_size=3, stride=1, padding=0) 
        self.cnn3 = nn.Conv2d(in_channels=18, out_channels=24, kernel_size=3, stride=1, padding=0) 
        self.fc1 = nn.Linear(24 * 9 * 9+comp, 2) 
        self.softmax = nn.Softmax(1)
    def forward(self, x,x2):
        # Convolution 1 49
        out = self.batchnorm(x)
        out = self.cnn1(x) # 48
        out = self.relu(out)
        out = self.maxpool(out) #24
        out = self.cnn2(out) #22
        out = self.relu(out) 
        out = self.maxpool(out) #11
        out = self.cnn3(out) #9
        out = self.relu(out)
        out = out.view(out.size(0), -1)
        print(out.shape)
        out = torch.cat([out,x2],1)
        # Linear function (readout)
        out = self.fc1(out)
        out = self.softmax(out)
        return out